# 골빈해커 3분딥러닝 텐서플로맛 Ch7

## 7.2

In [5]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot = True)


Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


X 의 첫번째 차원 : None. 입력값의 개수 <br>
마지막차원 : 1. 특징의 개수. MNIST는 회색조 이미지라 채널이 1개라서. <br>
Y : 출력값 10개 class <br>
keep_prob : dropout을 위해

In [7]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

CNN 계층 구현 <br>
먼저 3 by 3 커널을 가진 convolution 계층을 만든다. <br>

In [10]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev = 0.01))
L1 = tf.nn.conv2d(X, W1, strides = [1,1,1,1], padding = 'SAME')
L1 = tf.nn.relu(L1)

Pooling 계층

In [12]:
L1 = tf.nn.max_pool(L1, ksize= [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

두번째 계층

In [13]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev= 0.01))
L2 = tf.nn.conv2d(L1, W2, strides = [1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)
tf.nn.max_pool(L2, ksize = [1,2,2,1], strides = [1,2,2,1], padding = 'SAME')

<tf.Tensor 'MaxPool_1:0' shape=(?, 7, 7, 64) dtype=float32>

추출한 특징들을 이용해 10개의 분류를 만들어내는 계층

10개의 분류는 1차원 배열이므로 차원을 줄이는 단계를 거쳐야 한다. 

In [16]:
W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev = 0.01))
L3 = tf.reshape(L2, [-1, 7*7*64]) # 그냥 7*7*64 크기의 1차원 계층으로 만든다. 
L3 = tf.matmul(L3,W3) # L3을 최종 출력값의 중간단계인 256개의 뉴런으로 연결하는 신경망. Fully connected
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob) # 데이터의 overfitting을 막아주는 dropout 사용

L3의 출력값 256개를 받아 최종 출력값 0~9의 레이블을 갖는 10개의 출력값을 만든다. 

In [17]:
W4 = tf.Variable(tf.random_normal([256, 10], stddev = 0.01))
model = tf.matmul(L3, W4)

loss function과 AdamOptimizer를 사용해 최적화

In [19]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = model, labels = Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [ ]:
batch_xs.shape